#### Draft code for project 3
### Apriori Algorithm on NYC Property Assessment Dataset

In [352]:
import numpy as np
import pandas as pd

import itertools

In [347]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

#### data preprocessing
rearange as "transaction - items"

In [353]:
file = 'data/INTEGRATED-DATASET.csv'
df = pd.read_csv(file)

# random sample 5k samples
df = df.sample(5000)
df.head(5)

In [ ]:
df.rename({'Community.Board': 'Community_Board', 'Council.District': 'Council_District','Census.Tract':'Census_Tract'}, axis=1, inplace=True)
df.Community_Board = df.Community_Board.astype(str)
df.Council_District = df.Council_District.astype(str)
df['Building_Area'] = df['BLDFRONT']*df['BLDDEPTH']
df['Lot_Area'] = df['LTFRONT']*df['LTDEPTH']

df.Census_Tract = df.Census_Tract.astype(int)

In [ ]:
df.BIN.unique()

array([2085687, 3204172, 1063310, ..., 1039997, 1026888, 4433321])

In [ ]:
'''take a few fields for test'''
df = df.loc[:, ['BORO', 'STORIES', 'FULLVAL', 'YEAR', 'LOT', 'BLDGCL', 'AVTOT2', 'Community_Board', 'Council_District']]
df.head(10)

2072


,BORO,STORIES,FULLVAL,YEAR,LOT,BLDGCL,AVTOT2,Community_Board,Council_District,Lot_Area,Building_Area,Census_Tract,AVLAND2,EXLAND2,EXTOT2,EXCD2
51600,2,6.0,2603000.0,2012/13,1348,D4,1207710.0,208,11,32375,12075,337,333000.0,53309.0,53309,1019
49551,1,6.0,2260000.0,2012/13,36,D4,1016550.0,112,7,32860,32550,247,331200.0,331200.0,672306,1015
32453,1,5.0,522000.0,2014/15,67,C6,208260.0,111,5,2500,2150,15802,25668.0,25668.0,86666,1017
66011,2,2.0,480600.0,2011/12,49,M1,198954.0,204,16,4554,3960,199,14508.0,14508.0,198954,1101
28625,3,6.0,11338000.0,2015/16,40,D4,4977720.0,315,48,69300,64600,588,1150830.0,267076.0,267076,1015
20795,4,6.0,3595000.0,2016/17,13,D4,1490130.0,402,26,15000,12600,183,296550.0,72280.0,72280,1015
49618,2,6.0,1878000.0,2012/13,22,D4,721800.0,205,15,10000,8800,23502,40158.0,40158.0,374742,5129
24859,1,20.0,46963000.0,2015/16,27,D4,18152460.0,108,5,20196,18432,136,3559500.0,378173.0,378173,1015
46244,4,3.0,1255000.0,2013/14,27,C6,548640.0,408,24,20500,10205,77904,178200.0,22435.0,22435,1017
11239,2,6.0,3262000.0,2017/18,203,D4,1316340.0,208,11,16875,10665,30701,176220.0,49487.0,49487,1017


In [ ]:
# find proper splitting points for categorization
df.quantile([.33, .66], axis = 0)

,BORO,STORIES,FULLVAL,LOT,AVTOT2,Lot_Area,Building_Area,Census_Tract,AVLAND2,EXLAND2,EXTOT2,EXCD2
0.33,1.0,5.0,2177300.0,19.0,887366.7,7980.0,5250.0,179.0,175243.5,62504.62,85000.23,1015.0
0.66,3.0,7.0,8717320.0,45.0,3548624.2,18878.6,12825.0,542.0,765000.0,146929.54,203523.72,1017.0


In [ ]:
'''categorize field values'''

# borough
BORO_cond = [
    (df['BORO'] == 1),
    (df['BORO'] == 2),
    (df['BORO'] == 3),
    (df['BORO'] == 4),
    (df['BORO'] == 5)
]
BORO_cat = ['bor_1', 'bor_2', 'bor_3', 'bor_4', 'bor_5']
df['BORO'] = np.select(BORO_cond, BORO_cat)

# stories
STORIES_cond = [
    (df['STORIES'] <= 6),
    ((df['STORIES'] > 6) & (df['STORIES'] <= 12)),
    (df['STORIES'] > 12)
]
STORIES_cat = ['str_1', 'str_2', 'str_3']
df['STORIES'] = np.select(STORIES_cond, STORIES_cat)

# market value
FULLVAL_cond = [
    (df['FULLVAL'] <= 82000),
    ((df['FULLVAL'] > 82000) & (df['FULLVAL'] <= 1384000)),
    (df['FULLVAL'] > 1384000)
]
FULLVAL_cat = ['val_1', 'val_2', 'val_3']
df['FULLVAL'] = np.select(FULLVAL_cond, FULLVAL_cat)

# year
YEAR_cond = [
    (df['YEAR'] <= '2012/13'),
    ((df['YEAR'] > '2012/13') & (df['YEAR'] <= '2015/16')),
    (df['YEAR'] > '2015/16')
]
YEAR_cat = ['yrs_1', 'yrs_2', 'yrs_3']
df['YEAR'] = np.select(YEAR_cond, YEAR_cat)

# LOT
LOT_cond = [
    (df['LOT'] <= 61),
    ((df['LOT'] > 61) & (df['LOT'] <= 1126)),
    (df['LOT'] > 1126)
]
LOT_cat = ['lot_1', 'lot_2', 'lot_3']
df['LOT'] = np.select(LOT_cond, LOT_cat)

# AVTOT2
AVTOT2_cond = [
    (df['AVTOT2'] <= 35000),
    ((df['AVTOT2'] > 35000) & (df['AVTOT2'] <= 560000)),
    (df['AVTOT2'] > 560000)
]
AVTOT2_cat = ['lot_1', 'lot_2', 'lot_3']
df['AVTOT2'] = np.select(AVTOT2_cond, AVTOT2_cat)


# PSOTCODE
# df['POSTCODE'] = df['POSTCODE'].astype(str)
# df['POSTCODE'] = 'pos_' + df['POSTCODE']

# BLDGCL (building class)
df['BLDGCL'] = 'bcl_' + df['BLDGCL']

df['Community_Board'] = 'cmb_' + df['Community_Board']

df['Council_District'] = 'cds_' + df['Council_District']


df.head(10)

,BORO,STORIES,FULLVAL,YEAR,LOT,BLDGCL,AVTOT2,Community_Board,Council_District,Lot_Area,Building_Area,Census_Tract,AVLAND2,EXLAND2,EXTOT2,EXCD2
51600,bor_2,str_1,val_3,yrs_1,lot_3,bcl_D4,lot_3,cmb_208,cds_11,lotarea_2,buildingarea_2,census_1,avland2_3,exland2_2,extot2_2,extot2_2
49551,bor_1,str_1,val_3,yrs_1,lot_1,bcl_D4,lot_3,cmb_112,cds_7,lotarea_2,buildingarea_2,census_1,avland2_3,exland2_3,extot2_3,extot2_3
32453,bor_1,str_1,val_2,yrs_2,lot_2,bcl_C6,lot_2,cmb_111,cds_5,lotarea_1,buildingarea_1,census_3,avland2_2,exland2_1,extot2_2,extot2_2
66011,bor_2,str_1,val_2,yrs_1,lot_1,bcl_M1,lot_2,cmb_204,cds_16,lotarea_1,buildingarea_1,census_1,avland2_2,exland2_1,extot2_3,extot2_3
28625,bor_3,str_1,val_3,yrs_2,lot_1,bcl_D4,lot_3,cmb_315,cds_48,lotarea_3,buildingarea_3,census_1,avland2_3,exland2_3,extot2_3,extot2_3
20795,bor_4,str_1,val_3,yrs_3,lot_1,bcl_D4,lot_3,cmb_402,cds_26,lotarea_2,buildingarea_2,census_1,avland2_3,exland2_2,extot2_2,extot2_2
49618,bor_2,str_1,val_3,yrs_1,lot_1,bcl_D4,lot_3,cmb_205,cds_15,lotarea_1,buildingarea_1,census_3,avland2_2,exland2_1,extot2_3,extot2_3
24859,bor_1,str_3,val_3,yrs_2,lot_1,bcl_D4,lot_3,cmb_108,cds_5,lotarea_2,buildingarea_2,census_1,avland2_3,exland2_3,extot2_3,extot2_3
46244,bor_4,str_1,val_2,yrs_2,lot_1,bcl_C6,lot_2,cmb_408,cds_24,lotarea_2,buildingarea_2,census_3,avland2_3,exland2_1,extot2_1,extot2_1
11239,bor_2,str_1,val_3,yrs_3,lot_2,bcl_D4,lot_3,cmb_208,cds_11,lotarea_2,buildingarea_2,census_3,avland2_3,exland2_1,extot2_1,extot2_1


In [ ]:
# inspect categorization result
print(df.BORO.value_counts())
print(df.FULLVAL.value_counts())
print(df.STORIES.value_counts())
print(df.YEAR.value_counts())
print(df.LOT.value_counts())
print(df.BLDGCL.value_counts())
# print(df.POSTCODE.value_counts())
print(df.AVTOT2.value_counts())

bor_1    883
bor_3    506
bor_4    469
bor_2    198
bor_5     16
Name: BORO, dtype: int64
val_3    1545
val_2     483
val_1      44
Name: FULLVAL, dtype: int64
str_1    1345
str_3     472
str_2     255
Name: STORIES, dtype: int64
yrs_2    708
yrs_3    701
yrs_1    663
Name: YEAR, dtype: int64
lot_1    1657
lot_2     389
lot_3      26
Name: LOT, dtype: int64
bcl_D4    1300
bcl_C6     319
bcl_M1     118
bcl_R2      65
bcl_M9      51
bcl_C1      47
bcl_R4      31
bcl_M3      23
bcl_D0      23
bcl_C5      13
bcl_C7      11
bcl_D1       9
bcl_W2       6
bcl_R9       6
bcl_P7       4
bcl_K4       4
bcl_O8       3
bcl_E1       3
bcl_M2       2
bcl_W3       2
bcl_I9       2
bcl_O1       2
bcl_O9       2
bcl_C4       2
bcl_D7       2
bcl_O3       2
bcl_F1       2
bcl_Y2       2
bcl_E9       2
bcl_F9       1
bcl_G2       1
bcl_D3       1
bcl_G9       1
bcl_I6       1
bcl_P5       1
bcl_D6       1
bcl_W5       1
bcl_K1       1
bcl_M4       1
bcl_C8       1
bcl_Y1       1
bcl_D8       1
bcl_Z4    

In [ ]:
# full item list
items = [_ for _ in df.BORO.unique()]
items.extend([_ for _ in df.FULLVAL.unique()])
items.extend([_ for _ in df.STORIES.unique()])
items.extend([_ for _ in df.YEAR.unique()])
items.extend([_ for _ in df.LOT.unique()])
items.extend([_ for _ in df.BLDGCL.unique()])
# items.extend([_ for _ in df.POSTCODE.unique()])
items.extend([_ for _ in df.AVTOT2.unique()])
items.extend([_ for _ in df.Community_Board.unique()])
items.extend([_ for _ in df.Council_District.unique()])

items

['bor_2',
 'bor_1',
 'bor_3',
 'bor_4',
 'bor_5',
 'val_3',
 'val_2',
 'val_1',
 'str_1',
 'str_3',
 'str_2',
 'yrs_1',
 'yrs_2',
 'yrs_3',
 'lot_3',
 'lot_1',
 'lot_2',
 'bcl_D4',
 'bcl_C6',
 'bcl_M1',
 'bcl_R2',
 'bcl_C1',
 'bcl_O3',
 'bcl_M3',
 'bcl_D0',
 'bcl_F1',
 'bcl_E9',
 'bcl_D1',
 'bcl_D8',
 'bcl_M9',
 'bcl_P5',
 'bcl_R4',
 'bcl_M4',
 'bcl_C5',
 'bcl_O8',
 'bcl_C7',
 'bcl_Y2',
 'bcl_E1',
 'bcl_R9',
 'bcl_W2',
 'bcl_D7',
 'bcl_P7',
 'bcl_Y1',
 'bcl_C8',
 'bcl_O1',
 'bcl_C4',
 'bcl_K4',
 'bcl_K1',
 'bcl_I9',
 'bcl_W5',
 'bcl_D6',
 'bcl_M2',
 'bcl_I6',
 'bcl_W3',
 'bcl_D3',
 'bcl_G2',
 'bcl_O9',
 'bcl_F9',
 'bcl_G9',
 'bcl_Z4',
 'lot_3',
 'lot_2',
 'lot_1',
 'cmb_208',
 'cmb_112',
 'cmb_111',
 'cmb_204',
 'cmb_315',
 'cmb_402',
 'cmb_205',
 'cmb_108',
 'cmb_408',
 'cmb_302',
 'cmb_206',
 'cmb_303',
 'cmb_211',
 'cmb_105',
 'cmb_109',
 'cmb_309',
 'cmb_404',
 'cmb_312',
 'cmb_406',
 'cmb_106',
 'cmb_306',
 'cmb_301',
 'cmb_407',
 'cmb_403',
 'cmb_411',
 'cmb_107',
 'cmb_104',
 'c

In [ ]:
# convert the transaction dataframe into list of sets

trans = []

for index, row in df.iterrows():
    trans.append(set(sorted([row['BORO'], row['FULLVAL'], row['STORIES'], 
                            row['YEAR'], row['LOT'], row['BLDGCL'], row['AVTOT2'],
                            row['Community_Board'], row['Council_District'],row['Lot_Area'],row['Building_Area'],row['Census_Tract'],row['AVLAND2'],row['EXLAND2'],row['EXTOT2'],row['EXCD2']])))

#df = df.loc[:, ['BORO', 'STORIES', 'FULLVAL', 'YEAR', 'LOT', 'BLDGCL', 'AVTOT2', 'Community_Board', 'Council_District','Lot_Area','Building_Area','Census_Tract','AVLAND2','EXLAND2','EXTOT2','EXCD2']]


trans

#### Apriori

In [ ]:
# test combination function
# tmp = ['a', 'b', 'c', 'd']
# for comb in itertools.combinations(tmp, 3):
    # print(comb)

In [ ]:
# test prune()
# L_pre = [('a', 'b', 'c'), ('a', 'b', 'd'), ('b', 'c', 'e'), ('a', 'b', 'f'), ('b', 'd', 'k')]
# C = [('a', 'b', 'c', 'd'), ('a', 'b', 'c', 'f'), ('a', 'b', 'd', 'f'), ('a', 'b', 'c', 'e')]
# 
# prune(C, L_pre)

In [ ]:
# test join()
# L = [['a', 'b', 'c'], ['a', 'b', 'd'], ['b', 'c', 'e'], ['a', 'b', 'f'], ['b', 'd', 'k']]
# L = [('a'), ('b'), ('c')]
# L = [('bor_1', 'val_1', 'yrs_1'), ('bor_1', 'val_1', 'yrs_2'), ('bor_1', 'val_1', 'zzz_2')]
# join(L)

In [350]:
def join(L):
    """
    Follows the implementation in paper's section 2.1.1
    input: list of item lists of length k
    output: list of item lists of length k+1, i.e., candidate item lists of length k+1, or C_{k+1} in paper's notation
    """

    L_next = []

    # special case of 1-itemset
    if len(L[0]) == 1:
        for i in range(len(L)):
            for j in range(i+1, len(L)):
                # avoid redundancy, e.g., L[i] = ('bor_1', 'val_2', 'yrs_1')  L[j] = ('bor_1', 'val_2', 'yrs_2')
                # generating (..., yrs_1, yrs_2) is nonsense since yrs_1 and yrs_2 are the same category 
                if L[i][-1][:3] == L[j][-1][:3]:
                    # print('skip...', L[i][-1][:3])
                    continue
                # add to L_2-itemset
                tmp = list(L[i])
                tmp.extend(L[j])
                L_next.append(tuple(tmp))
        return L_next

    for i in range(len(L)):
        for j in range(i+1, len(L)):
            if L[i][:-1] == L[j][:-1] and L[i][-1] < L[j][-1]:
                # print(L[i][:-1])
                # print(L[j][:-1])
                # print()
                # print(L[i][-1])
                # print(L[j][-1])
                # print()
                # avoid redundancy, e.g., L[i] = ('bor_1', 'val_2', 'yrs_1')  L[j] = ('bor_1', 'val_2', 'yrs_2')
                # generating (..., yrs_1, yrs_2) is nonsense since yrs_1 and yrs_2 are the same category 
                if L[i][-1][:3] == L[j][-1][:3]:
                    # print('skip...', L[i][-1][:3])
                    continue
                # add to L_k+1
                tmp = list(L[i])
                tmp.append(L[j][-1])
                L_next.append(tmp)
    return L_next


def prune(C, L_pre):
    """
    Follows the implementation in paper's section 2.1.1
    input: 
        C: a list of candidate item lists of length k
        L_pre: k-itemset
    output: pruned list of item lists, i.e., L_k+1
    """

    for itemsets in C:
        # check every (k-1)-subset of itemsets is in L_k-1
        # print('itemset: ', itemsets)
        for subset in itertools.combinations(itemsets, len(itemsets)-1):
            # print('subset: ', subset)
            if subset not in L_pre:
                # remove itemset from C
                # print('attempt to remove ', itemsets)
                C.remove(itemsets)
                break

    return C


def apriori_gen(L):
    """
    input: current list of k-itemsets
    output: next, i.e., k+1-itemsets
    """

    Candidates = join(L)

    # let's just make sure the items in each itemset is in sorted order
    res = []
    tmp = prune(Candidates, L)
    for x in tmp:
        res.append(tuple(sorted(x)))

    return res


In [ ]:
# t = [('a_1', 'c_2', 'b_3'), ('z', 'i')]
# k = []
# for i in t:
#     k.append(tuple(sorted(i)))
# k

In [ ]:
tmp = {('a', 'b'): 25, ('b', 'c'): 33, ('d', 'e'): 44}

len(tmp.keys())

3

In [351]:
"""main loop"""

# input params
min_sup = 0.02
min_conf = 0.5

min_sup_cnt = min_sup * len(trans)

####################################
# L = {
#   1: {'item_1': #sup,
#       'item_2': #sup,
#        ...      
#       },
#   2: {('item_1', 'item_2'): #sup,
#       ...
#       },
#   3: {('item_1', 'item_4', 'item_6): #sup,
#        ...
#       },
#   ...
# }
####################################
L = dict() 

'''initialization: init L1 (1-itemsets and counts)'''
print('Initializing L1...\n')
L[1] = dict()
for item in items:
    cnt = 0
    for row in trans:
        if item in row:
            cnt += 1
    L[1][tuple([item])] = cnt
print('Initialization...Done\n')

k = 2
# while L_(k-1) is not empty, execute the loop
while len(L[k-1].keys()) > 0:

    Ck = apriori_gen(list(L[k-1].keys()))
    print('--------------------')
    print('k = ', k)
    # print('Ck = \n', Ck)

    # one pass counting for Ck
    # construct hash table (dict)
    cnt_tb = dict()
    for c in Ck:    # each c is a tuple of items
        cnt_tb[c] = 0

    # count
    for tran in trans:
        for tup in itertools.combinations(tran, k): 
            tup = tuple(sorted(tup))
            if tup in set(cnt_tb.keys()):
                cnt_tb[tup] += 1

    # eliminate tuples < min_sup, i.e., only add those >= min_sup
    L[k] = dict()
    for tup, cnt in cnt_tb.items():
        # print(tup)
        # print(cnt)
        # print()
        if cnt >= min_sup_cnt:
            L[k][tup] = cnt

    k += 1
    print(L)



NameError: name 'trans' is not defined

In [ ]:
cnt_tb

{}

In [ ]:
L

{1: {('bor_2',): 198,
  ('bor_1',): 883,
  ('bor_3',): 506,
  ('bor_4',): 469,
  ('bor_5',): 16,
  ('val_3',): 1545,
  ('val_2',): 483,
  ('val_1',): 44,
  ('str_1',): 1345,
  ('str_3',): 472,
  ('str_2',): 255,
  ('yrs_1',): 663,
  ('yrs_2',): 708,
  ('yrs_3',): 701,
  ('lot_3',): 1560,
  ('lot_1',): 1699,
  ('lot_2',): 768,
  ('bcl_D4',): 1300,
  ('bcl_C6',): 319,
  ('bcl_M1',): 118,
  ('bcl_R2',): 65,
  ('bcl_C1',): 47,
  ('bcl_O3',): 2,
  ('bcl_M3',): 23,
  ('bcl_D0',): 23,
  ('bcl_F1',): 2,
  ('bcl_E9',): 2,
  ('bcl_D1',): 9,
  ('bcl_D8',): 1,
  ('bcl_M9',): 51,
  ('bcl_P5',): 1,
  ('bcl_R4',): 31,
  ('bcl_M4',): 1,
  ('bcl_C5',): 13,
  ('bcl_O8',): 3,
  ('bcl_C7',): 11,
  ('bcl_Y2',): 2,
  ('bcl_E1',): 3,
  ('bcl_R9',): 6,
  ('bcl_W2',): 6,
  ('bcl_D7',): 2,
  ('bcl_P7',): 4,
  ('bcl_Y1',): 1,
  ('bcl_C8',): 1,
  ('bcl_O1',): 2,
  ('bcl_C4',): 2,
  ('bcl_K4',): 4,
  ('bcl_K1',): 1,
  ('bcl_I9',): 2,
  ('bcl_W5',): 1,
  ('bcl_D6',): 1,
  ('bcl_M2',): 2,
  ('bcl_I6',): 1,
  ('bcl_W

#### Generate association rules

In [ ]:
rules = []

for k, k_itemsest in L.items():
    if k == 1:
        # trivial case
        continue
    
    # compute confidence for every permutation in every itemset tuple
    print('k = ', k)
    for item_tup, sup in k_itemsest.items():
        for item in item_tup:
            print(item, item_tup)
            # rule = item -> every other items in item_tup
            # conf = sup(item_tup) / sup(item)
            right_items = set(item_tup) - {item}    # doesn't matter if right item is sorted bc the computation doesn't use it
            item_sup = L[1][tuple([item])]
            item_tup_sup = sup
            conf = item_tup_sup / item_sup

            # print(item, ' ------> ', right_items, 'conf: ', conf, 'sup: ', sup, '\n')
            if conf >= min_conf:
                # item => right_items, conf, support
                rules.append(([item], list(right_items), conf, sup))



k =  2
bor_2 ('bor_2', 'val_3')
val_3 ('bor_2', 'val_3')
bor_2 ('bor_2', 'val_2')
val_2 ('bor_2', 'val_2')
bor_2 ('bor_2', 'str_1')
str_1 ('bor_2', 'str_1')
bor_2 ('bor_2', 'yrs_1')
yrs_1 ('bor_2', 'yrs_1')
bor_2 ('bor_2', 'yrs_2')
yrs_2 ('bor_2', 'yrs_2')
bor_2 ('bor_2', 'yrs_3')
yrs_3 ('bor_2', 'yrs_3')
bor_2 ('bor_2', 'lot_3')
lot_3 ('bor_2', 'lot_3')
bor_2 ('bor_2', 'lot_1')
lot_1 ('bor_2', 'lot_1')
bor_2 ('bor_2', 'lot_2')
lot_2 ('bor_2', 'lot_2')
bcl_D4 ('bcl_D4', 'bor_2')
bor_2 ('bcl_D4', 'bor_2')
bor_2 ('bor_2', 'cmb_208')
cmb_208 ('bor_2', 'cmb_208')
bor_2 ('bor_2', 'cds_11')
cds_11 ('bor_2', 'cds_11')
bor_2 ('bor_2', 'lotarea_2')
lotarea_2 ('bor_2', 'lotarea_2')
bor_2 ('bor_2', 'lotarea_1')
lotarea_1 ('bor_2', 'lotarea_1')
bor_2 ('bor_2', 'buildingarea_2')
buildingarea_2 ('bor_2', 'buildingarea_2')
bor_2 ('bor_2', 'buildingarea_1')
buildingarea_1 ('bor_2', 'buildingarea_1')
bor_2 ('bor_2', 'census_1')
census_1 ('bor_2', 'census_1')
bor_2 ('bor_2', 'census_3')
census_3 ('bor_2

In [ ]:
rules = sorted(rules, key=lambda x: x[2], reverse=True)
rules

[(['cmb_208'], ['bor_2'], 1.0, 59),
 (['cds_11'], ['bor_2'], 1.0, 70),
 (['cmb_108'], ['bor_1'], 1.0, 263),
 (['cmb_109'], ['bor_1'], 1.0, 65),
 (['cmb_106'], ['bor_1'], 1.0, 100),
 (['cmb_107'], ['bor_1'], 1.0, 177),
 (['cmb_104'], ['bor_1'], 1.0, 48),
 (['cmb_102'], ['bor_1'], 1.0, 50),
 (['cmb_110'], ['bor_1'], 1.0, 56),
 (['cds_7'], ['bor_1'], 1.0, 89),
 (['cds_5'], ['bor_1'], 1.0, 112),
 (['cds_4'], ['bor_1'], 1.0, 242),
 (['cds_6'], ['bor_1'], 1.0, 153),
 (['cds_3'], ['bor_1'], 1.0, 83),
 (['cds_2'], ['bor_1'], 1.0, 73),
 (['cds_9'], ['bor_1'], 1.0, 70),
 (['cmb_315'], ['bor_3'], 1.0, 62),
 (['cmb_312'], ['bor_3'], 1.0, 73),
 (['cmb_314'], ['bor_3'], 1.0, 42),
 (['cds_48'], ['bor_3'], 1.0, 68),
 (['cds_44'], ['bor_3'], 1.0, 51),
 (['cds_39'], ['bor_3'], 1.0, 62),
 (['cds_43'], ['bor_3'], 1.0, 49),
 (['cmb_406'], ['bor_4'], 1.0, 89),
 (['cmb_407'], ['bor_4'], 1.0, 107),
 (['cmb_403'], ['bor_4'], 1.0, 63),
 (['cds_25'], ['bor_4'], 1.0, 70),
 (['cds_29'], ['bor_4'], 1.0, 109),
 (['c

In [ ]:
%reset -f